In [1]:
import pickle
import pandas as pd
import pathlib
import sys
from spacy.tokens import Doc
from collections import Counter

In [2]:
spacy_pipeline_parent_path = pathlib.Path.cwd().parent.parent.parent.parent
sys.path.append(str(spacy_pipeline_parent_path))

from spacy_pipeline import pipeline_setup

vocab = pipeline_setup.get_vocab()
pipeline_setup.set_all_extensions()

pickle_dir = str(pathlib.Path.cwd().parent)
pickle_file = 'label_news_200_docs_stanza.pkl'

with open(pickle_dir + '/' + pickle_file, 'rb') as f:
    bytes_data = pickle.load(f)
    all_docs_stanza = [[Doc(vocab).from_bytes(doc_bytes) for doc_bytes in docs] for docs in bytes_data]

2023-03-25 19:00:07 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-03-25 19:00:09 INFO: Loading these models for language: zh-hant (Traditional_Chinese):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2023-03-25 19:00:09 INFO: Using device: cpu
2023-03-25 19:00:09 INFO: Loading: tokenize
2023-03-25 19:00:09 INFO: Loading: pos
2023-03-25 19:00:09 INFO: Loading: lemma
2023-03-25 19:00:10 INFO: Loading: depparse
2023-03-25 19:00:10 INFO: Done loading processors!


In [3]:
all_docs_stanza

[[民進黨立委范雲表示將提出修法，讓女性有資格服兵役。女性是否納義務役引起各界討論，對女性服兵役是否影響未來民進黨的選情？台南市長黃偉哲今天低調表示，希望多所考量，並多徵詢各方意見。,
  黃偉哲今天參加台南市元旦升旗典禮，針對立委范雲要修法讓女性有資格服兵役一事，他受訪表示，尊重立委的提案權，有很多國家女性是不用服兵役，因為立法院是合議制，要經過多數立委同意才有辦法修法，對於范雲的主張，他表示尊重。,
  對於范雲的主張，會不會影響未來民進黨的選情？黃偉哲表示，「這個喔？無從評估！」「其實在台灣，已有女性志願役，如要改成義務役，大家還是…」黃偉哲欲言又止，他說，「希望多所考量，並多徵詢各方意見。」],
 [蔡英文總統今發表2023年元旦談話，揭示執政團隊4項重要任務：照顧人民生活、維持經濟動能、守護國家安全、善盡區域責任，呼籲大家團結一心，讓國家更進步、繁榮、平安、幸福。,
  蔡英文指出，受到烏俄戰爭、全球通膨、威權擴張的持續衝擊，國際上許多主要機構都預測，2023年全球經濟正持續轉弱，各國不僅要面對經濟、金融的劇烈波動，能源、糧食危機反覆出現的高度不確定性，也讓經濟結構、成長動能備受考驗，台灣也不例外。,
  蔡英文說明，執政團隊昨國安高層會議提出包括減輕人民負擔、穩定民生物價、調整產業體質、維持經濟動能等四大目標，並提出第一波經濟民生因應措施的7項方案。攸關國家長遠發展的六大核心戰略產業，總統府和行政院，也將共同進行國家戰略產業發展的階段檢討，並召開諮詢會議，邀請民間企業及公協會，共同參與，包括半導體及資通訊產業、綠能產業、基礎建設相關產業、以及國防產業等，歡迎產業提供建議、策略，進一步鞏固台灣在全球供應鏈的關鍵角色。,
  在稅收超徵部份，蔡英文說明，扣除分配給地方政府的700億元，中央稅收3800億元餘裕將朝三個方向規劃使用，首先加碼1000億元撥補勞保及健保基金財務缺口，同時進行電價補貼、其次是以1000億元支應第一波七項方案，第三是預留一定財源以備不時之需，並在財政許可範圍內適時研議全民共享的可行方案。,
  蔡英文說，近來有人主張皆大歡喜，將這筆錢平均分給每一個人，但在全球經濟高度不確定的此刻，作為負責任的政府，我們必須要未雨綢繆，把寶貴的資源，優先協助高風險的國人與產業，來維持國家因應危機的韌性，「這個決定不容易，但為了國家，我們必須這樣做，也要

In [4]:
all_docs_stanza[115][0].spans

{'ckip_ner': [美國海軍第七艦隊, 十三日, 十二日, 南海, 一, 三, 美軍, 中方, 兩, 中國, 十四日, 山東號航艦艦隊, 南海], 'opinion_label': [美國海軍第七艦隊, 宣布, 尼米茲號航空母艦打擊群十二日起在南海展開多項操練行動，該打擊群包括一艘巡洋艦與三艘驅逐艦。, 美軍官員, 透露, 中方派出兩艘軍艦跟蹤監視；中國軍方隨後於十四日宣布，山東號航艦艦隊近日在南海進行實兵實戰化對抗演練。], 'coreference_label': []}

In [34]:
counter = Counter()

for docs in all_docs_stanza:
    for doc in docs:
        counter['paragraph'] += 1
        if 'opinion_label' in doc.spans:
            if len(doc.spans['opinion_label']) > 0:
                counter['opinioned_paragraph'] += 1
            for span_group in doc.spans['opinion_label']: 
                counter[span_group.label_] += 1
counter

Counter({'paragraph': 1105,
         'opinioned_paragraph': 807,
         'OPINION_OPR': 1008,
         'OPINION_SRC': 991,
         'OPINION_SEG': 1059})

In [6]:
991 + 1095

2086

# OPINION_OPR analysis

In [11]:
df_opr_feature = pd.DataFrame(columns=['label_id', 'span_label', 'num_of_token', 'stanza_pos', 'stanza_dep', 'ckip_pos',  'token_text'])

for docs in all_docs_stanza:
    for doc in docs:
        if 'opinion_label' in doc.spans:
            for span in doc.spans['opinion_label']:
                if span.label_ == 'OPINION_OPR':
                    new_row = pd.DataFrame(
                        columns=df_opr_feature.columns,
                        index=[0],
                        data={
                            'label_id': str(span._.label_id),
                            'span_label': str(span.label_),
                            'num_of_token': len(span),
                            'stanza_pos': str([token.pos_ for token in span]),
                            'stanza_dep': str([token.dep_ for token in span]),
                            'ckip_pos': str([token.tag_ for token in span]),
                            'token_text': str([token.text for token in span])
                        })
                    if len(new_row) > 1:
                        new_row = new_row[:1]
                    df_opr_feature = pd.concat([df_opr_feature, new_row], ignore_index=True)

df_opr_feature.to_csv('OPINION_OPR_feature.csv', index=False)

df_opr_feature



Counter({'OPINION_OPR': 1008})


,label_id,span_label,num_of_token,stanza_pos,stanza_dep,ckip_pos,token_text
0,['0RJsZQqAb5'],OPINION_OPR,1,['VERB'],['root'],['VE'],['表示']
1,['sVC9O4AHXb'],OPINION_OPR,1,['VERB'],['root'],['VE'],['表示']
2,['0APo8lGJia'],OPINION_OPR,1,['VERB'],['root'],['VE'],['表示']
3,['36Pj4jY4ht'],OPINION_OPR,1,['VERB'],['root'],['VE'],['表示']
4,['WhFjddsiu5'],OPINION_OPR,1,['VERB'],['root'],['VE'],['說']
...,...,...,...,...,...,...,...
1003,['SSKSvg0XqM'],OPINION_OPR,1,['VERB'],['root'],['VE'],['稱']
1004,['uklXhrgxil'],OPINION_OPR,1,['VERB'],['root'],['VE'],['表示']
1005,['TKtHSoCDUz'],OPINION_OPR,1,['VERB'],['parataxis'],['VE'],['說']
1006,['0vH1b54af-'],OPINION_OPR,1,['VERB'],['root'],['VE'],['表示']


In [26]:
(df_opr_feature['stanza_pos'] + ' | ' + df_opr_feature['ckip_pos']+ ' | ' + df_opr_feature['stanza_dep']).value_counts()[:15]

['VERB'] | ['VE'] | ['root']                           595
['VERB'] | ['VE'] | ['advcl']                           90
['VERB'] | ['VE'] | ['parataxis']                       86
['VERB'] | ['VC'] | ['root']                            42
['VERB'] | ['VE'] | ['ccomp']                           25
['VERB'] | ['VC'] | ['advcl']                           12
['VERB'] | ['VE'] | ['xcomp']                            9
['VERB'] | ['VK'] | ['advcl']                            9
['VERB'] | ['VK'] | ['root']                             8
['VERB'] | ['VA'] | ['root']                             7
['VERB'] | ['VE'] | ['acl']                              6
['VERB'] | ['VA'] | ['advcl']                            6
['VERB'] | ['VC'] | ['parataxis']                        4
['NOUN'] | ['VE'] | ['nsubj']                            4
['ADV', 'VERB'] | ['VE', 'VE'] | ['advmod', 'root']      4
dtype: int64

In [25]:
(df_opr_feature['stanza_pos'] + ' | ' + df_opr_feature['ckip_pos']+ ' | ' + df_opr_feature['stanza_dep']).value_counts()[:15] / len(df_opr_feature) * 100

['VERB'] | ['VE'] | ['root']                           59.027778
['VERB'] | ['VE'] | ['advcl']                           8.928571
['VERB'] | ['VE'] | ['parataxis']                       8.531746
['VERB'] | ['VC'] | ['root']                            4.166667
['VERB'] | ['VE'] | ['ccomp']                           2.480159
['VERB'] | ['VC'] | ['advcl']                           1.190476
['VERB'] | ['VE'] | ['xcomp']                           0.892857
['VERB'] | ['VK'] | ['advcl']                           0.892857
['VERB'] | ['VK'] | ['root']                            0.793651
['VERB'] | ['VA'] | ['root']                            0.694444
['VERB'] | ['VE'] | ['acl']                             0.595238
['VERB'] | ['VA'] | ['advcl']                           0.595238
['VERB'] | ['VC'] | ['parataxis']                       0.396825
['NOUN'] | ['VE'] | ['nsubj']                           0.396825
['ADV', 'VERB'] | ['VE', 'VE'] | ['advmod', 'root']     0.396825
dtype: float64

In [43]:
(1008 - (df_opr_feature['stanza_pos'] + ' | ' + df_opr_feature['ckip_pos']+ ' | ' + df_opr_feature['stanza_dep']).value_counts()[:15].sum()) / len(df_opr_feature) * 100

10.01984126984127

In [32]:
df_opr_feature[(df_opr_feature['stanza_pos'] == "['NOUN']") & (df_opr_feature['ckip_pos'] == "['VE']")]['token_text'].unique()

array(["['說明']", "['問侯']", "['認為']", "['直言']", "['說']", "['強調']"],
      dtype=object)

In [44]:
(df_opr_feature['stanza_pos'] + ' | ' + df_opr_feature['ckip_pos']).value_counts()[:15]

['VERB'] | ['VE']                  812
['VERB'] | ['VC']                   62
['VERB'] | ['VK']                   18
['VERB'] | ['VA']                   17
['NOUN'] | ['VE']                   12
['VERB'] | ['VF']                    7
['ADV', 'VERB'] | ['VE', 'VE']       7
['VERB'] | ['VH']                    6
['VERB'] | ['D']                     5
['NOUN'] | ['VC']                    4
['VERB'] | ['Na']                    4
['VERB'] | ['VJ']                    4
['VERB'] | ['Nb']                    3
['VERB', 'VERB'] | ['VE', 'VE']      3
['VERB'] | ['VG']                    3
dtype: int64

In [14]:
df_opr_feature['num_of_token'].value_counts()

1    973
2     34
3      1
Name: num_of_token, dtype: int64

In [37]:
df_opr_feature['num_of_token'].value_counts() / df_opr_feature['num_of_token'].value_counts().sum() * 100

1    96.527778
2     3.373016
3     0.099206
Name: num_of_token, dtype: float64

In [15]:
df_opr_feature[df_opr_feature['num_of_token'] == 2].groupby(['token_text', 'stanza_pos', 'ckip_pos', 'stanza_dep']).count()

,,,,label_id,span_label,num_of_token
token_text,stanza_pos,ckip_pos,stanza_dep,,,
"['並期', '許']","['VERB', 'PROPN']","['Cbb', 'Nb']","['advcl', 'nmod']",1,1,1
"['他祝', '福國']","['PROPN', 'PROPN']","['Nh', 'Na']","['nmod', 'nmod']",1,1,1
"['做', '結']","['VERB', 'VERB']","['VA', 'Na']","['root', 'xcomp']",1,1,1
"['問', '說']","['VERB', 'VERB']","['VE', 'VE']","['advcl', 'root']",1,1,1
"['問', '道']","['VERB', 'VERB']","['VE', 'VE']","['ccomp', 'xcomp']",1,1,1
"['嗆', '言']","['VERB', 'VERB']","['VE', 'VE']","['advcl', 'root']",1,1,1
"['嗨', '喊']","['PART', 'VERB']","['I', 'VE']","['nsubj', 'advcl']",1,1,1
"['回', '覆']","['VERB', 'VERB']","['VD', 'VD']","['advcl', 'mark']",1,1,1
"['坦', '言']","['ADV', 'VERB']","['VE', 'VE']","['advmod', 'advcl']",1,1,1


In [16]:
df_opr_feature[df_opr_feature['num_of_token'] == 2]['stanza_pos'].value_counts()

['VERB', 'VERB']      9
['ADV', 'VERB']       9
['VERB', 'PROPN']     3
['NOUN', 'VERB']      3
['PROPN', 'PROPN']    2
['ADJ', 'VERB']       2
['PART', 'VERB']      2
['PROPN', 'PART']     1
['PROPN', 'VERB']     1
['VERB', 'NOUN']      1
['NOUN', 'PART']      1
Name: stanza_pos, dtype: int64

In [17]:
df_opr_feature[df_opr_feature['num_of_token'] == 2]['ckip_pos'].value_counts()

['VE', 'VE']     13
['VC', 'VC']      5
['Dfa', 'VC']     2
['VA', 'Na']      2
['VD', 'VD']      1
['Na', 'VH']      1
['Nc', 'VC']      1
['D', 'VE']       1
['VK', 'VK']      1
['VG', 'VG']      1
['Nh', 'Na']      1
['I', 'VE']       1
['VA', 'VH']      1
['Cbb', 'Nb']     1
['Na', 'VC']      1
['VE', 'VC']      1
Name: ckip_pos, dtype: int64

In [18]:
df_opr_feature[df_opr_feature['num_of_token'] == 2]['token_text'].value_counts()

['坦', '言']      7
['痛', '批']      2
['澄', '清']      1
['更怒', '轟']     1
['問', '說']      1
['做', '結']      1
['砲', '轟苗']     1
['直', '指']      1
['士葆痛', '批']    1
['並期', '許']     1
['游怒', '嗆']     1
['嗨', '喊']      1
['他祝', '福國']    1
['斥', '為']      1
['期', '盼']      1
['祝', '賀']      1
['重澄', '清']     1
['邁期', '許']     1
['問', '道']      1
['學痛', '批']     1
['宇怒', '批']     1
['慧大', '酸']     1
['笑', '說']      1
['笑', '稱']      1
['嗆', '言']      1
['回', '覆']      1
['留', '言']      1
Name: token_text, dtype: int64

In [19]:
df_opr_feature[df_opr_feature['num_of_token'] == 3]['token_text'].value_counts()

['不', '諱', '言']    1
Name: token_text, dtype: int64

# relation analysis

In [20]:
columns=['from_label_id', 'from_label', 'from_token_text',  'from_token_stanza_pos', 'from_token_stanza_dep', 'from_token_ckip_pos', 
             'to_label_id',
             'to_label',
             'to_token_text',
             'to_token_stanza_pos',
             'to_token_stanza_dep',
             'to_token_ckip_pos',
             'relation_label', 
            'direction']
df_relation_feature = pd.DataFrame(columns=columns)

for docs in all_docs_stanza:
    for doc in docs:
        if 'opinion_label' in doc.spans:
            for span_group in doc.spans['opinion_label']:
                if span_group.label_ == 'OPINION_SRC' or span_group.label_ == 'OPINION_SEG':
                    for src_or_seg_token in span_group:
                        if 'OPINION_OPR' in src_or_seg_token.head._.label_type:
                            relation_label = [relation['relation_label'] for relation in src_or_seg_token._.relation_label if relation['from_id'] in src_or_seg_token.head._.label_id and relation['to_id'] in src_or_seg_token._.label_id]
                            relation_label = relation_label[0] if len(relation_label) > 0 else None
                            data = {
                                    'from_label_id': src_or_seg_token.head._.label_id,
                                    'from_label': src_or_seg_token.head._.label_type,
                                    'from_token_text': src_or_seg_token.head.text,
                                    'from_token_stanza_pos': src_or_seg_token.head.pos_,
                                    'from_token_stanza_dep': src_or_seg_token.head.dep_,
                                    'from_token_ckip_pos': src_or_seg_token.head.tag_,

                                    'to_label_id': ';'.join(src_or_seg_token._.label_id),
                                    'to_label': ';'.join(src_or_seg_token._.label_type),
                                    'to_token_text': src_or_seg_token.text,
                                    'to_token_stanza_pos': src_or_seg_token.pos_,
                                    'to_token_stanza_dep': src_or_seg_token.dep_,
                                    'to_token_ckip_pos': src_or_seg_token.tag_,
                                    
                                    'relation_label': relation_label,
                                    'direction': 'left' if src_or_seg_token.i < src_or_seg_token.head.i else 'right'
                                }
                            
                            new_row = pd.DataFrame(
                                columns=columns,
                                data=data)
                            df_relation_feature = pd.concat([df_relation_feature, new_row], ignore_index=True)

df_relation_feature.to_csv('OPINION_cross_label_dep_feature.csv', index=False)
df_relation_feature

,from_label_id,from_label,from_token_text,from_token_stanza_pos,from_token_stanza_dep,from_token_ckip_pos,to_label_id,to_label,to_token_text,to_token_stanza_pos,to_token_stanza_dep,to_token_ckip_pos,relation_label,direction
0,0RJsZQqAb5,OPINION_OPR,表示,VERB,root,VE,KYl-FS1X6n,OPINION_SRC,立委,NOUN,nsubj,Na,OPR->SRC,left
1,0RJsZQqAb5,OPINION_OPR,表示,VERB,root,VE,MG1umndr9X,OPINION_SEG,讓,VERB,xcomp,VL,OPR->SEG,right
2,0RJsZQqAb5,OPINION_OPR,表示,VERB,root,VE,MG1umndr9X,OPINION_SEG,。,PUNCT,punct,PERIODCATEGORY,OPR->SEG,right
3,sVC9O4AHXb,OPINION_OPR,表示,VERB,root,VE,8xVRxD3iiH,OPINION_SRC,市長,NOUN,nsubj,Na,OPR->SRC,left
4,sVC9O4AHXb,OPINION_OPR,表示,VERB,root,VE,0Z0Z9aIACs,OPINION_SEG,徵詢,VERB,ccomp,VE,OPR->SEG,right
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3270,0vH1b54af-,OPINION_OPR,表示,VERB,root,VE,Bi2eaTZgnJ,OPINION_SEG,。,PUNCT,punct,PERIODCATEGORY,OPR->SEG,right
3271,lnmD6NHBNK,OPINION_OPR,指出,VERB,root,VE,KTZD9t77cu,OPINION_SRC,陳,PROPN,nsubj,Nb,OPR->SRC,left
3272,lnmD6NHBNK,OPINION_OPR,指出,VERB,root,VE,xErc3GMcqK,OPINION_SEG,不辜,VERB,ccomp,D,OPR->SEG,right
3273,lnmD6NHBNK,OPINION_OPR,指出,VERB,root,VE,xErc3GMcqK,OPINION_SEG,邁進,VERB,ccomp,VCL,OPR->SEG,right


In [21]:
df_relation_feature[df_relation_feature['to_token_stanza_pos'] == 'PUNCT']['to_token_text'].value_counts()

。    618
，    328
？     15
」     10
！      4
；      3
：      1
Name: to_token_text, dtype: int64

In [53]:
df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SRC')]['to_token_stanza_dep'].value_counts()

/var/folders/q1/yjtyjl7n4631vjyys7_0q0k40000gn/T/ipykernel_57190/2030365308.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SRC')]['to_token_stanza_dep'].value_counts()


nsubj        638
advcl         19
advmod        13
nmod           8
aux            8
nmod:tmod      2
case           2
compound       2
mark           1
cop            1
Name: to_token_stanza_dep, dtype: int64

In [55]:
sum = df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SRC')]['to_token_stanza_dep'].value_counts().sum()
df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SRC')]['to_token_stanza_dep'].value_counts() / sum * 100

/var/folders/q1/yjtyjl7n4631vjyys7_0q0k40000gn/T/ipykernel_57190/1269026998.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sum = df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SRC')]['to_token_stanza_dep'].value_counts().sum()
/var/folders/q1/yjtyjl7n4631vjyys7_0q0k40000gn/T/ipykernel_57190/1269026998.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SRC')]['to_token_stanza_dep'].value_counts() / sum * 100


nsubj        91.930836
advcl         2.737752
advmod        1.873199
nmod          1.152738
aux           1.152738
nmod:tmod     0.288184
case          0.288184
compound      0.288184
mark          0.144092
cop           0.144092
Name: to_token_stanza_dep, dtype: float64

# OPR and SEG

In [56]:
df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SEG')]['to_token_stanza_dep'].value_counts()

/var/folders/q1/yjtyjl7n4631vjyys7_0q0k40000gn/T/ipykernel_57190/2065481550.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SEG')]['to_token_stanza_dep'].value_counts()


punct        665
ccomp        662
parataxis    221
xcomp         55
obj           50
advcl         15
obl            3
nsubj          3
conj           2
mark           2
nmod:tmod      1
mark:rel       1
Name: to_token_stanza_dep, dtype: int64

In [57]:
sum = df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SEG')]['to_token_stanza_dep'].value_counts().sum()
df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SEG')]['to_token_stanza_dep'].value_counts() / sum * 100

/var/folders/q1/yjtyjl7n4631vjyys7_0q0k40000gn/T/ipykernel_57190/1764702016.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sum = df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SEG')]['to_token_stanza_dep'].value_counts().sum()
/var/folders/q1/yjtyjl7n4631vjyys7_0q0k40000gn/T/ipykernel_57190/1764702016.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_relation_feature[['from_label_id', 'from_label', 'to_label_id', 'to_label', 'to_token_stanza_dep', 'direction']].drop_duplicates()[(df_relation_feature['from_label']=='OPINION_OPR') & (df_relation_feature['to_label']=='OPINION_SEG')]['to_token_stanza_dep'].value_counts() / sum * 100


punct        39.583333
ccomp        39.404762
parataxis    13.154762
xcomp         3.273810
obj           2.976190
advcl         0.892857
obl           0.178571
nsubj         0.178571
conj          0.119048
mark          0.119048
nmod:tmod     0.059524
mark:rel      0.059524
Name: to_token_stanza_dep, dtype: float64

# Pronounce

In [62]:
df_pronounce_feature = pd.DataFrame(columns=['label_id', 'span_label', 'num_of_token', 'stanza_pos', 'stanza_dep', 'ckip_pos',  'token_text'])

for docs in all_docs_stanza:
    for doc in docs:
        if 'coreference_label' in doc.spans:
            for span in doc.spans['coreference_label']:
                if span.label_ == 'Pronoun':
                    new_row = pd.DataFrame(
                        columns=df_pronounce_feature.columns,
                        index=[0],
                        data={
                            'label_id': str(span._.label_id),
                            'span_label': str(span.label_),
                            'num_of_token': len(span),
                            'stanza_pos': str([token.pos_ for token in span]),
                            'stanza_dep': str([token.dep_ for token in span]),
                            'ckip_pos': str([token.tag_ for token in span]),
                            'token_text': str([token.text for token in span])
                        })
                    if len(new_row) > 1:
                        new_row = new_row[:1]
                    df_pronounce_feature = pd.concat([df_pronounce_feature, new_row], ignore_index=True)

df_pronounce_feature.to_csv('Pronounce_feature.csv', index=False)
df_pronounce_feature

,label_id,span_label,num_of_token,stanza_pos,stanza_dep,ckip_pos,token_text
0,"['k5K_dfPFam', '29Xdoo-Vez']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['他']
1,"['odsYxV1YeQ', 'XWGmDJV_2H']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['他']
2,"['A-oARVLfAG', 'lP0ruQtbwz']",Pronoun,1,['PROPN'],['nsubj'],['Nb'],['侯']
3,"['uSHdkyecLi', 'ehTqe-DTXv']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['她']
4,"['2QT8KcHE1J', 'UO6mPn_slh']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['她']
...,...,...,...,...,...,...,...
140,"['OVZkiH7SVf', '2ov452P2An']",Pronoun,1,['NOUN'],['nsubj'],['Na'],['總統']
141,"['Qxf7Vo3cxE', 'VxtpjXs-vx']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['他']
142,"['sp248r0UVJ', '8Qd4J5uGep']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['他']
143,"['jTmtVjE5J1', 'RJOFH7Fhef']",Pronoun,1,['PRON'],['nsubj'],['Nh'],['他']


In [63]:
(df_pronounce_feature['stanza_pos'] + ' | ' + df_pronounce_feature['ckip_pos']+ ' | ' + df_pronounce_feature['stanza_dep']).value_counts()[:15]

['PRON'] | ['Nh'] | ['nsubj']                                                 76
['PROPN'] | ['Nb'] | ['nsubj']                                                26
['NOUN'] | ['Na'] | ['nsubj']                                                  8
['PROPN', 'NOUN'] | ['Nb', 'Na'] | ['nmod', 'nsubj']                           7
['PROPN', 'PROPN'] | ['Nb', 'Na'] | ['nsubj', 'flat:name']                     3
['NUM', 'NOUN', 'NOUN'] | ['Neu', 'Nf', 'Na'] | ['nummod', 'clf', 'nsubj']     2
['PROPN', 'NOUN'] | ['Nb', 'Na'] | ['nsubj', 'nsubj']                          2
['PROPN', 'PART', 'NOUN'] | ['Nb', 'Na', 'Na'] | ['nmod', 'case', 'nsubj']     2
['DET', 'NOUN'] | ['Nes', 'Na'] | ['det', 'nsubj']                             2
['PROPN'] | ['Nh'] | ['nmod']                                                  1
['ADJ'] | ['Nb'] | ['csubj']                                                   1
['VERB'] | ['Nb'] | ['advcl']                                                  1
['DET', 'NOUN', 'NOUN'] | ['

In [66]:
sum = (df_pronounce_feature['stanza_pos'] + ' | ' + df_pronounce_feature['ckip_pos']+ ' | ' + df_pronounce_feature['stanza_dep']).value_counts()[:15].sum()

In [67]:
(df_pronounce_feature['stanza_pos'] + ' | ' + df_pronounce_feature['ckip_pos']+ ' | ' + df_pronounce_feature['stanza_dep']).value_counts()[:15] / sum * 100

['PRON'] | ['Nh'] | ['nsubj']                                                 56.716418
['PROPN'] | ['Nb'] | ['nsubj']                                                19.402985
['NOUN'] | ['Na'] | ['nsubj']                                                  5.970149
['PROPN', 'NOUN'] | ['Nb', 'Na'] | ['nmod', 'nsubj']                           5.223881
['PROPN', 'PROPN'] | ['Nb', 'Na'] | ['nsubj', 'flat:name']                     2.238806
['NUM', 'NOUN', 'NOUN'] | ['Neu', 'Nf', 'Na'] | ['nummod', 'clf', 'nsubj']     1.492537
['PROPN', 'NOUN'] | ['Nb', 'Na'] | ['nsubj', 'nsubj']                          1.492537
['PROPN', 'PART', 'NOUN'] | ['Nb', 'Na', 'Na'] | ['nmod', 'case', 'nsubj']     1.492537
['DET', 'NOUN'] | ['Nes', 'Na'] | ['det', 'nsubj']                             1.492537
['PROPN'] | ['Nh'] | ['nmod']                                                  0.746269
['ADJ'] | ['Nb'] | ['csubj']                                                   0.746269
['VERB'] | ['Nb'] | ['advcl']   

# Coreference

In [70]:


df_coreference_feature = pd.DataFrame(columns=['label_id', 'span_label', 'num_of_token', 'stanza_pos', 'stanza_dep', 'ckip_pos',  'token_text'])

for docs in all_docs_stanza:
    for doc in docs:
        if 'coreference_label' in doc.spans:
            for span in doc.spans['coreference_label']:
                if span.label_ == 'Coreference':
                    new_row = pd.DataFrame(
                        columns=df_coreference_feature.columns,
                        index=[0],
                        data={
                            'label_id': str(span._.label_id),
                            'span_label': str(span.label_),
                            'num_of_token': len(span),
                            'stanza_pos': str([token.pos_ for token in span]),
                            'stanza_dep': str([token.dep_ for token in span]),
                            'ckip_pos': str([token.tag_ for token in span]),
                            'token_text': str([token.text for token in span])
                        })
                    if len(new_row) > 1:
                        new_row = new_row[:1]
                    df_coreference_feature = pd.concat([df_coreference_feature, new_row], ignore_index=True)

df_coreference_feature.to_csv('Pronounce_feature.csv', index=False)
df_coreference_feature

,label_id,span_label,num_of_token,stanza_pos,stanza_dep,ckip_pos,token_text
0,['mYhP4PWEAI'],Coreference,3,"['PROPN', 'PROPN', 'PROPN']","['nsubj', 'flat:name', 'flat:name']","['Nb', 'Na', 'Na']","['黃', '偉', '哲']"
1,['OhRV83GY9-'],Coreference,3,"['PROPN', 'PROPN', 'PROPN']","['nsubj', 'flat:name', 'flat:name']","['Nb', 'Na', 'Na']","['黃', '偉', '哲']"
2,"['2Modo1hgPb', 'shQd7Tqzzo']",Coreference,2,"['NOUN', 'PROPN']","['nsubj', 'appos']","['Nb', 'D']","['侯友', '宜']"
3,"['Qm4op9Ji0-', 'z73VeOjGbj']",Coreference,7,"['NOUN', 'PART', 'PROPN', 'PART', 'NOUN', 'PRO...","['compound', 'nmod', 'compound', 'nmod', 'nsub...","['Nb', 'Na', 'Nc', 'Na', 'Na', 'Nb', 'Nb']","['社民', '黨', '台北', '市', '議員', '苗', '博雅']"
4,['duMpgmhrx6'],Coreference,2,"['PROPN', 'PROPN']","['nsubj', 'flat:name']","['Nb', 'Nb']","['王', '鴻薇']"
...,...,...,...,...,...,...,...
102,"['KiswkapaPL', 'dbK30_jYf8']",Coreference,4,"['PROPN', 'PROPN', 'PART', 'NOUN']","['nmod', 'compound', 'nmod', 'nsubj']","['Nb', 'Nb', 'Nb', 'Na']","['蔡', '英', '文', '總統']"
103,['2XSZYzfV-G'],Coreference,6,"['PROPN', 'PART', 'VERB', 'PART', 'PROPN', 'PR...","['compound', 'nmod', 'compound', 'nsubj:pass',...","['Nb', 'Na', 'Na', 'Na', 'Nb', 'Nb']","['民進', '黨', '候選', '人', '吳', '怡農']"
104,['DCV4FO6o3M'],Coreference,3,"['PROPN', 'PROPN', 'NOUN']","['nsubj:pass', 'flat:name', 'advmod']","['Nb', 'Nb', 'Na']","['吳', '怡', '農今']"
105,"['AR2dzLfQtl', 'DZIo7niJF0']",Coreference,3,"['PROPN', 'PROPN', 'VERB']","['nsubj', 'flat:name', 'advcl']","['Nb', 'Nb', 'VH']","['黃', '英', '慈']"


In [71]:
(df_coreference_feature['stanza_pos'] + ' | ' + df_coreference_feature['ckip_pos']+ ' | ' + df_coreference_feature['stanza_dep']).value_counts()[:15]

['PROPN', 'PROPN'] | ['Nb', 'Nb'] | ['nsubj', 'flat:name']                                                                           23
['PROPN'] | ['Nb'] | ['nsubj']                                                                                                       10
['PROPN', 'PROPN'] | ['Nb', 'Na'] | ['nsubj', 'flat:name']                                                                            6
['PROPN', 'PROPN', 'PROPN'] | ['Nb', 'Na', 'Na'] | ['nsubj', 'flat:name', 'flat:name']                                                3
['PROPN'] | ['Nb'] | ['nmod']                                                                                                         3
['NUM', 'NOUN', 'NOUN', 'PART', 'NOUN'] | ['Neu', 'Nf', 'Nb', 'Na', 'Na'] | ['nummod', 'clf', 'compound', 'nmod', 'nsubj']            2
['PROPN', 'PROPN'] | ['Nb', 'Nb'] | ['conj', 'flat:name']                                                                             2
['PROPN', 'PROPN'] | ['Nb', 'Nb'] | ['appos', 'f

In [73]:
sum = (df_coreference_feature['stanza_pos'] + ' | ' + df_coreference_feature['ckip_pos']+ ' | ' + df_coreference_feature['stanza_dep']).value_counts()[:15].sum()
(df_coreference_feature['stanza_pos'] + ' | ' + df_coreference_feature['ckip_pos']+ ' | ' + df_coreference_feature['stanza_dep']).value_counts()[:15] / sum * 100

['PROPN', 'PROPN'] | ['Nb', 'Nb'] | ['nsubj', 'flat:name']                                                                           37.704918
['PROPN'] | ['Nb'] | ['nsubj']                                                                                                       16.393443
['PROPN', 'PROPN'] | ['Nb', 'Na'] | ['nsubj', 'flat:name']                                                                            9.836066
['PROPN', 'PROPN', 'PROPN'] | ['Nb', 'Na', 'Na'] | ['nsubj', 'flat:name', 'flat:name']                                                4.918033
['PROPN'] | ['Nb'] | ['nmod']                                                                                                         4.918033
['NUM', 'NOUN', 'NOUN', 'PART', 'NOUN'] | ['Neu', 'Nf', 'Nb', 'Na', 'Na'] | ['nummod', 'clf', 'compound', 'nmod', 'nsubj']            3.278689
['PROPN', 'PROPN'] | ['Nb', 'Nb'] | ['conj', 'flat:name']                                                                             3.278689

In [80]:
counter = Counter()
for docs in all_docs_stanza:
    for doc in docs:
        if 'coreference_label' in doc.spans:
            for span in doc.spans['coreference_label']:
                if span.label_ == 'Coreference':
                    counter['Coreference'] += 1
                    counter[str([ent.label_ for ent in span.ents])] += 1
                    # print(span.text, span.ents, [ent.label_ for ent in span.ents])

counter

Counter({'Coreference': 107,
         "['PERSON']": 82,
         "['ORG', 'GPE', 'PERSON']": 1,
         '[]': 5,
         "['ORG', 'PERSON']": 8,
         "['GPE']": 1,
         "['GPE', 'ORG', 'PERSON']": 1,
         "['GPE', 'PERSON']": 4,
         "['ORG']": 3,
         "['CARDINAL', 'ORG']": 2})